In [1]:
import torch
torch.__version__

'2.2.1'

In [2]:
x = torch.randn(4,4)
x

tensor([[-2.5277, -1.5839,  1.1411, -0.2757],
        [-1.0695, -2.0831, -1.1950,  1.6209],
        [ 2.8176, -0.6958, -0.3485, -0.0895],
        [ 0.6868, -1.1868,  0.0324, -0.7502]])

In [3]:
a = torch.ones(5)
b = a.numpy()
b

array([1., 1., 1., 1., 1.], dtype=float32)

In [4]:
x = torch.randn(3,4,requires_grad=True)
x

tensor([[-0.6466, -0.0385, -0.2858,  0.1096],
        [-0.1337, -0.6948,  0.8531, -1.1416],
        [-1.0525,  0.0899, -0.5280,  0.7684]], requires_grad=True)

In [5]:
b = torch.randn(3,4,requires_grad=True)
b

tensor([[-0.9489,  0.8014, -0.5200,  1.4198],
        [ 0.2935,  0.1470, -0.1978, -0.1214],
        [-0.6636,  0.1992, -0.1156,  0.8006]], requires_grad=True)

In [12]:
t = x + b
t

tensor([[-1.5955,  0.7628, -0.8059,  1.5294],
        [ 0.1598, -0.5478,  0.6553, -1.2630],
        [-1.7162,  0.2891, -0.6436,  1.5691]], grad_fn=<AddBackward0>)

In [14]:
y = t.sum()
y

tensor(-1.6064, grad_fn=<SumBackward0>)

In [18]:
y.backward()
b.grad

tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])

In [20]:
x.requires_grad, b.requires_grad, t.requires_grad

(True, True, True)

In [25]:
x = torch.rand(1)
b = torch.rand(1,requires_grad=True)
w = torch.rand(1,requires_grad=True)
y = w*x
z = y+b
z

tensor([0.7681], grad_fn=<AddBackward0>)

In [28]:
x.requires_grad, b.requires_grad, w.requires_grad, y.requires_grad

(False, True, True, True)

In [31]:
z.backward(retain_graph=True)

In [32]:
w.grad

tensor([0.6272])

In [44]:
b.grad

tensor([3.])

線性回歸
---

Step 1: 定義數據集
--

In [46]:
import numpy as np

x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32) # np.array couldn't be train in NN.
x_train = x_train.reshape(-1,1) # trans to arrray
x_train.shape

# trun np_array into array 

(11, 1)

In [94]:
y_values = [2*i for i in x_values]
y_train = np.array(x_values, dtype=np.float32)
y_train = x_train.reshape(-1,1)
y_train.shape

(11, 1)

Step 2: 定義模型
--

In [47]:
import torch
import torch.nn as nn

In [68]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim , output_dim):
        super(LinearRegressionModel,self).__init__()
        self.linear = nn.Linear(input_dim, output_dim) # layers to be used
    def forward(self, x):
        out = self.linear(x) # how to walk
        return out

In [79]:
input_dim = 1
output_dim = 1
model = LinearRegressionModel(input_dim, output_dim)
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

Step 3: 定義參數
---

In [95]:
epochs = 1000
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

Step 4: 開始訓練
---

In [81]:
for epoch in range(epochs):
    epoch += 1
    # 轉換 np_array
    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_train)
    
    # 梯度清零時迭代
    optimizer.zero_grad()
    
    # 向前傳播
    output = model(inputs)
    
    # 計算損失
    loss = criterion(output, labels)
    
    # 反向傳播
    loss.backward()
    
    # 更新權重參數
    optimizer.step()
    
    if epoch%50 ==0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))    

epoch 50, loss 0.009800647385418415
epoch 100, loss 0.005589930806308985
epoch 150, loss 0.003188276896253228
epoch 200, loss 0.0018184765940532088
epoch 250, loss 0.0010371903190389276
epoch 300, loss 0.0005915748188272119
epoch 350, loss 0.0003374131629243493
epoch 400, loss 0.00019244504801463336
epoch 450, loss 0.0001097639324143529
epoch 500, loss 6.260559166548774e-05
epoch 550, loss 3.570803164620884e-05
epoch 600, loss 2.0366664102766663e-05
epoch 650, loss 1.1616771189437713e-05
epoch 700, loss 6.625246442126809e-06
epoch 750, loss 3.7788913687109016e-06
epoch 800, loss 2.1553810256591532e-06
epoch 850, loss 1.2292916835576762e-06
epoch 900, loss 7.012276910245419e-07
epoch 950, loss 3.9999346768127e-07
epoch 1000, loss 2.2810550603935553e-07


Step 5: 預測模型結果
---

In [96]:
predicted = model(torch.from_numpy(x_train).requires_grad_()).data.numpy() # turn to np.array
predicted

array([[-3.7340565e-06],
       [ 9.9999684e-01],
       [ 1.9999975e+00],
       [ 2.9999981e+00],
       [ 3.9999986e+00],
       [ 4.9999990e+00],
       [ 6.0000000e+00],
       [ 7.0000005e+00],
       [ 8.0000010e+00],
       [ 9.0000019e+00],
       [ 1.0000002e+01]], dtype=float32)

Step 6: 模型保存與讀取
---
> 生產環境中需要邊訓練邊保存好的權重

In [86]:
torch.save(model.state_dict(),'model.pk1')

In [90]:
model.load_state_dict(torch.load('model.pk1'))

<All keys matched successfully>

使用GPU訓練
---
> 把 數據 與 模型 導入
cuda

In [93]:
# model傳到 GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
model.to(device)


for epoch in range(epochs):
    epoch += 1
    # 參數傳到 GPU
    inputs = torch.from_numpy(x_train).to(device)
    labels = torch.from_numpy(y_train).to(device)
    
    # 梯度清零時迭代
    optimizer.zero_grad()
    
    # 向前傳播
    output = model(inputs)
    
    # 計算損失
    loss = criterion(output, labels)
    
    # 反向傳播
    loss.backward()
    
    # 更新權重參數

    optimizer.step()
    
    if epoch%50 ==0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))    

epoch 50, loss 1.301679191101357e-07
epoch 100, loss 7.422138281754087e-08
epoch 150, loss 4.2271654621117705e-08
epoch 200, loss 2.4142394750015228e-08
epoch 250, loss 1.3762433859199064e-08
epoch 300, loss 7.846784555454178e-09
epoch 350, loss 4.489085547731975e-09
epoch 400, loss 2.547222655024939e-09
epoch 450, loss 1.4561197803075743e-09
epoch 500, loss 8.245861216060746e-10
epoch 550, loss 4.728066271830755e-10
epoch 600, loss 2.7335358976365853e-10
epoch 650, loss 1.5524549701328993e-10
epoch 700, loss 9.028764541563206e-11
epoch 750, loss 5.2760049801259257e-11
epoch 800, loss 3.0272225698402266e-11
epoch 850, loss 1.8278312891029103e-11
epoch 900, loss 1.1109039035894774e-11
epoch 950, loss 6.610444396731863e-12
epoch 1000, loss 4.119225793797199e-12


重點整理
---
1. 在pytorch不需要自己定義反向傳播